### 1. Load Spark

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from geopy.distance import distance, great_circle
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString, Polygon
from shapely import wkt
import shapely.speedups
shapely.speedups.enable()

In [2]:
from pyspark.sql.types import *
import findspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('ITPT2_MAP').getOrCreate()

In [8]:
spark._jvm.org.apache.spark.sql.types.SqlGeometry.registerAll(spark._jsparkSession)

NameError: name 'spark' is not defined

### 2. Read data

Load the network

In [4]:
network_df = pd.read_csv('road_network.txt', delimiter = "\t", header=0)
network_df = network_df.rename(columns={'LINESTRING()': 'geometry', ' Speed (m/s)': 'Speed'})
network_df['geometry'] = network_df['geometry'].apply(wkt.loads)
network_gdf = gpd.GeoDataFrame(network_df, crs='epsg:4326')[['Edge ID', 'geometry']]
network_gdf.head()

,Edge ID,geometry
0,883991900000,"LINESTRING (-122.73232 47.88992, -122.73214 47..."
1,883991900001,"LINESTRING (-122.71108 47.87765, -122.71240 47..."
2,883991900002,"LINESTRING (-122.70742 47.87615, -122.70738 47..."
3,883991900003,"LINESTRING (-122.70742 47.87615, -122.70839 47..."
4,883991900004,"LINESTRING (-122.71533 47.88187, -122.71627 47..."


In [5]:
# Convert the geometry column to string/wkt 
network_gdf['wkt'] = pd.Series(
        map(lambda geom: str(geom.to_wkt()), network_gdf['geometry']),
        index=network_gdf.index, dtype='string')
network_gdf.drop(columns='geometry', inplace=True)
network_gdf.rename(columns={'Edge ID': 'Edge_ID'}, inplace=True)
network_gdf.head()

,Edge_ID,wkt
0,883991900000,LINESTRING (-122.7323189377780039 47.889919281...
1,883991900001,LINESTRING (-122.7110785245899933 47.877650856...
2,883991900002,LINESTRING (-122.7074199914929977 47.876151502...
3,883991900003,LINESTRING (-122.7074199914929977 47.876151502...
4,883991900004,LINESTRING (-122.7153298258780012 47.881869971...


In [6]:
network_schema = StructType([StructField('Edge_ID', StringType()),
                             StructField('wkt', StringType())])

In [13]:
import os
import sys

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] =  sys.executable

In [14]:
network_sdf = spark.createDataFrame(network_gdf, network_schema)
network_sdf.show(5)

Py4JJavaError: An error occurred while calling o127.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4) (ITEM-S76279.emea.msad.sopra executor driver): java.io.IOException: Cannot run program "python3": CreateProcess error=2, The system cannot find the file specified
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:165)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spar

Load the GPS data

In [8]:
gps_df = pd.read_csv('gps_data.txt', delimiter = "\t", header=0)
gps_df.drop(columns=['Unnamed: 4', 'Unnamed: 5'], inplace=True)
gps_gdf = gpd.GeoDataFrame(gps_df, 
                           geometry=gpd.points_from_xy(gps_df.Longitude, gps_df.Latitude),
                           crs='epsg:4326')
gps_gdf

,Date (UTC),Time (UTC),Latitude,Longitude,geometry
0,17-Jan-2009,20:27:37,47.667483,-122.107083,POINT (-122.10708 47.66748)
1,17-Jan-2009,20:27:38,47.667500,-122.107067,POINT (-122.10707 47.66750)
2,17-Jan-2009,20:27:39,47.667500,-122.107067,POINT (-122.10707 47.66750)
3,17-Jan-2009,20:27:40,47.667517,-122.107033,POINT (-122.10703 47.66752)
4,17-Jan-2009,20:27:41,47.667533,-122.106983,POINT (-122.10698 47.66753)
...,...,...,...,...,...
7526,17-Jan-2009,22:34:24,47.641450,-122.141917,POINT (-122.14192 47.64145)
7527,17-Jan-2009,22:34:25,47.641450,-122.141783,POINT (-122.14178 47.64145)
7528,17-Jan-2009,22:34:26,47.641433,-122.141650,POINT (-122.14165 47.64143)
7529,17-Jan-2009,22:34:27,47.641450,-122.141517,POINT (-122.14152 47.64145)


Preprocess the GPS data

In [14]:
def distance_between_row(data_gpd, sigma_z = 4.07):
    data_gpd['dist_prev'] = 0
    idx_range = list(np.arange(len(data_gpd)-1))
    
    i=0
    indexes_to_drop = []
    k=1
    
    while i+k < len(data_gpd):
        
        dist = great_circle(data_gpd.loc[i, 'geometry'].coords[0][::-1],
                            data_gpd.loc[i+k, 'geometry'].coords[0][::-1]).meters
        
        if dist >= 2 * sigma_z:
            data_gpd.loc[i+k, 'dist_prev'] = dist
            i = i+k
            k = 1
        else:
            indexes_to_drop.append(i+k)
            k += 1
          
    data_gpd_prepro = data_gpd.drop(indexes_to_drop) 
    # use data_gpd.drop(data_gpd.index[indexes_to_drop]) if index is not range(len())
    #print('Eliminated: {}%'.format(round(100*(1 - len(data_gpd_prepro)/len(data_gpd)),1)))
    return data_gpd_prepro
gps_gdf_prepro = distance_between_row(gps_gdf)

In [15]:
gps = gps_gdf_prepro.to_crs(epsg=32148)
gps.rename(columns={'Date (UTC)':'Date','Time (UTC)':'Time'}, inplace=True)
gps.reset_index(drop=True,inplace=True)
gps['datetime'] = pd.to_datetime(gps['Date'] + ' ' + gps['Time'])
gps.drop(columns=['Date', 'Time'], inplace=True)
gps.sort_values(by='datetime', inplace=True)

In [17]:
gps_sample = gps.iloc[:1000, :]

In [18]:
points = gps_sample['geometry']

### 3. Main function

In [16]:
def map_partitions_exec(param):
    def find_nearest_link_pt(iterator):
        """Find the nearest geometry among a list, measured from fixed point.

        Args:
            z: a shapely Point

        Returns:
            min_link: the nearest link ID
            min_dist: distance between the point to that link ID
        """
        for z in iterator:
        min_dist, min_idx = min((z.distance(geom), k) for (k, geom) in enumerate(network_gdf['geometry'].to_crs(epsg=32148)))

        min_link = network_gdf['Edge ID'].iloc[min_idx]

        return min_link, min_dist

In [19]:
bc_points = spark.sparkContext.broadcast(points)

In [21]:
params_spark = spark.sparkContext.parallelize(points,4)

In [ ]:
res = params_spark.mapPartitions(map_partitions_exec(bc_points.value))